## Heart Disease Dataset Variables (with Severity Notes) (Cheat sheet)

* **`ChestPainType`**: Type of chest pain.
    * `TA`: Typical Angina -> 🚨 **High Risk** (Classic heart attack warning)
    * `ATA`: Atypical Angina -> ⚠️ **Moderate Risk**
    * `NAP`: Non-Anginal Pain -> ✅ **Low Risk** (Likely not heart-related)
    * `ASY`: Asymptomatic -> ❓ **Silent/Sneaky** (Often dangerous in ML models)
* **`RestingBP`**: Resting blood pressure [mm Hg].
    * *Note:* Higher values -> ⚠️ **Worse** (Hypertension strain)
* **`Cholesterol`**: Serum cholesterol [mm/dl].
    * *Note:* Higher values -> ⚠️ **Worse** (Risk of blockage)
* **`FastingBS`**: Fasting blood sugar.
    * `0`: Normal (< 120 mg/dl) -> ✅ **Good**
    * `1`: High (> 120 mg/dl) -> ⚠️ **Diabetes Risk** (Damages vessels)
* **`RestingECG`**: Resting electrocardiogram results.
    * `Normal`: Normal -> ✅ **Good**
    * `ST`: ST-T abnormality -> ⚠️ **Bad** (Ischemia/Oxygen lack)
    * `LVH`: Left Ventricular Hypertrophy -> 🚨 **Severe** (Thickened heart muscle)
* **`MaxHR`**: Maximum heart rate achieved.
    * *Note:* Lower max achievable HR often indicates -> ⚠️ **Weaker Heart**
* **`ExerciseAngina`**: Exercise-induced angina.
    * `N`: No -> ✅ **Good**
    * `Y`: Yes -> 🚨 **Critical Flag** (Heart fails under stress)
* **`Oldpeak`**: ST depression induced by exercise vs rest.
    * *Note:* Higher value (>1.0) -> 🚨 **Tragic/Severe Ischemia**
* **`ST_Slope`**: The slope of the peak exercise ST segment.
    * `Up`: Upsloping -> ✅ **Healthy Response**
    * `Flat`: Flat -> ⚠️ **Warning**
    * `Down`: Downsloping -> ☠️ **Worst/Tragic** (Strongest sign of disease)

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from heart_failure_prediction.config import RAW_DATA_DIR
import heart_failure_prediction.plots as plots

In [ ]:
path = os.path.join(RAW_DATA_DIR, 'heart.csv')
heart = pd.read_csv(path)

In [ ]:
heart.head()

In [ ]:
heart.describe().T

### Encoding binary values as 0/1

In [ ]:
heart['ExerciseAngina'] = heart['ExerciseAngina'].map({'N': 0, 'Y': 1})
heart['Sex'] = heart['Sex'].map({'M': 0, 'F': 1})

### Data types

In [ ]:
heart.dtypes

After mapping binary values to 0/1 there are 3 categorical values: ChestPainType, RestingECG, ST_Slope

### Missing and duplicated values

In [ ]:
print(f'Null value count: {heart.isna().sum().sum()}')

In [ ]:
print(f'Duplicated values count: {heart.duplicated().sum()}')

There are no missing or duplicated values

### Data distribution

In [ ]:
plots.plot_distributions(heart)

In [ ]:
heart['HeartDisease'].value_counts(normalize=True) * 100

The distribution of the target value is more or less even

Cholesterol can't be 0! Suggests missing values.

In [ ]:
heart['Cholesterol'] = heart['Cholesterol'].replace({0: np.nan})

In [ ]:
print(f'Null value count: {heart.isna().sum().sum()}')

In [ ]:
heart['col_missing'] = heart['Cholesterol'].isna().astype(int)
print(heart.groupby('col_missing')['HeartDisease'].mean())
heart.drop('col_missing', axis=1, inplace=True)

Missing cholesterol suggests critical status of patient! Should add indicator while imputing later on.

### Looking for outliers

In [ ]:
num_cols = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
plt.figure(figsize=(15, 10))
for i, col in enumerate(num_cols):
    plt.subplot(2, 3, i + 1)
    sns.boxplot(x=heart[col])
    plt.title(col)
plt.show()

0 RestingBP is impossible. Dropping. Rest of the outliers are medically possible.

In [ ]:
zero_restingbp = heart.loc[heart['RestingBP'] == 0].index
heart.drop(zero_restingbp, axis=0, inplace=True)

Negative oldpeek suggests ST Elevation - it's still bad, worse than 0. For regression maybe should transform it to absolute value? For decision trees it will be ok.

### Correlation

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(
    heart.select_dtypes(np.number).corr(), annot=True, fmt='.2f', cmap='coolwarm'
)
plt.show()

There are no strongly correlated values. ExerciseAngina is the value strongest correlated with the target.

In [ ]:
plots.plot_categorical_countplots(heart, 'HeartDisease')

In [ ]:
print(heart.groupby('ChestPainType')['HeartDisease'].mean())

In [ ]:
print(heart.groupby('ST_Slope')['HeartDisease'].mean())

Patients with Asymptotic Chest Pain are much more likely to be in positive group. Suggests that patients can't feel the pain, ignore other symptomps and the disease can develop freely.

Non-flat ST_Slope patients have much greater risk of being in the positive group.